In [1]:
from __future__ import print_function
from glob import glob
from ROOT import vector, RDataFrame, RDF, TFile, TH1F, TH2F, gInterpreter, TMath
import ROOT
import sys
import yaml
from bff_processor.cpp_function import def_cpp
from bff_processor.utils import toVector, get_nEvents
from bff_processor.df_definitions import *
import pandas as pd
from time import perf_counter
import uproot

Welcome to JupyROOT 6.22/06


In [2]:
def_cpp()

True

In [3]:
#ROOT.ROOT.EnableImplicitMT()
RDFrame = RDataFrame

In [11]:
era = '2016'
if era == "2016":
    fname = "samplesCR_2016_Apr2020.yml"
    bDiscValue = 0.6321
if era == "2017":
    fname = "sampleCR_2017_v6_lo.yml"
    bDiscValue = 0.4941
if era == "2018":
    fname = "samplesCR_2018_v6_lo.yml"
    bDiscValue = 0.4184
outname = fname.replace('.yml','.root')
outname

'samplesCR_2016_Apr2020.root'

In [12]:
class sample_processor():
    def __init__(self,file_name,outname,bDiscValue,is_inclusive=0):
        #load config
        self.file_name = file_name
        with open(file_name,'r') as f:
            self.sample_dict = yaml.load(f, Loader=yaml.FullLoader)
        #setup outfile
        self.outname = outname
        self.out = TFile(outname, 'recreate')
        self.outdirs_dict = {}
        for sample in self.samples():
            name = sample['name']
            self.outdirs_dict[name] = self.out.mkdir(name)
        self.lumi = self.sample_dict['lumi']
        #get and write lumi info
        hlumi = TH1F("lumi", "lumi", 1, 0, 1)
        hlumi.SetDirectory(self.out)
        hlumi.SetBinContent(1, self.lumi)
        hlumi.Write()
        self.bDiscValue = bDiscValue
        self.is_inclusive = is_inclusive
    def samples(self):
        return self.sample_dict['samples']
    def sample_names(self):
        return [s['name'] for s in self.samples()]
    def close(self):
        self.out.Close()
    def __repr__(self):
        text_dict = {"fn":self.file_name,
                     "on":self.outname, 
                     "lumi":self.lumi,
                    "samples": self.sample_names()}
        return '''from {fn} to {on}\nlumi: {lumi}\nSamples {samples}'''.format(**text_dict)

In [13]:
sp = sample_processor(fname, outname, bDiscValue)
print(sp)

from samplesCR_2016_Apr2020.yml to samplesCR_2016_Apr2020.root
lumi: 35.5
Samples ['ZToEE_M_120_200', 'ZToEE_M_200_400', 'ZToEE_M_400_800', 'ZToEE_M_50_120', 'ZToEE_M_800_1400', 'ZToMuMu_M_120_200', 'ZToMuMu_M_200_400', 'ZToMuMu_M_400_800', 'ZToMuMu_M_50_120', 'ZToMuMu_M_800_1400', 'mc_santitop', 'mc_stop', 'mc_ttbar', 'mc_ww', 'mc_wz', 'mc_zz', 'BFFZprimeToMuMu_M_500', 'BFFZprimeToMuMu_M_500_dbs0p5', 'BFFZprimeToMuMu_M_500_dbs1p0', 'BFFZprimeToMuMu_M_350', 'BFFZprimeToMuMu_M_350_dbs0p5', 'BFFZprimeToMuMu_M_350_dbs1p0', 'BFFZprimeToMuMu_M_200', 'BFFZprimeToMuMu_M_200_dbs1p0', 'BFFZprimeToMuMu_M_200_dbs0p5', 'data_el', 'data_mu']


In [16]:
def create_regions(df, ismc):
    # create regions
    if int(ismc):
        JERC_var = ['nominal','jerUp','jerDown','jesUp','jesDown']
    else:
        JERC_var = ['nominal']
    rs = ["CR10", "CR11", "CR12", "CR13", "CR14", "CR20", "CR21", "CR22", "CR23", "CR24", "SR1", "SR2"]
    rs = [(r,var) for r in rs for var in JERC_var]
    for reg,var in rs:
        r = '{}_{}'.format(reg,var)
        HTLT_string = 'HTLT_{}'.format(var)
        RelMET_string = 'RelMET_{}'.format(var)
        SBM_string = 'SBM_{}'.format(var)

        HTLT,RelMET,SBM = -120,0.22,0
        if r[2] == "2":
            HTLT,RelMET,SBM = -60,0.22,150
        
        region_string = "{}pre_bff".format(r)
        format_dict = {"region_string":region_string, "r": r,"HTLT": HTLT,"RelMET": RelMET,"SBM": SBM,"HTLT_string": HTLT_string,"RelMET_string": RelMET_string,"SBM_string": SBM_string}
        regions.append((df.Filter("{r} && DiLepMass>54".format(**format_dict), region_string), format_dict))

        region_string = "{}".format(r)
        format_dict = {"region_string":region_string, "r": r,"HTLT": HTLT,"RelMET": RelMET,"SBM": SBM,"HTLT_string": HTLT_string,"RelMET_string": RelMET_string,"SBM_string": SBM_string}
        regions.append((df.Filter("{r} && DiLepMass>54 && {HTLT_string}<{HTLT} && {RelMET_string}<{RelMET} && {SBM_string}>{SBM}".format(**format_dict), region_string), format_dict))

        region_string = "{}_200_GeV_htlt_sig".format(r)
        format_dict = {"region_string":region_string, "r": r,"HTLT": HTLT,"RelMET": RelMET,"SBM": SBM,"HTLT_string": HTLT_string,"RelMET_string": RelMET_string,"SBM_string": SBM_string}
        regions.append((df.Filter("{r} && DiLepMass>54 && {RelMET_string}<{RelMET} && {SBM_string}>{SBM}".format(**format_dict), region_string), format_dict))

        region_string = "{}_200_GeV_sig".format(r)
        format_dict = {"region_string":region_string, "r": r,"HTLT": HTLT,"RelMET": RelMET,"SBM": SBM,"HTLT_string": HTLT_string,"RelMET_string": RelMET_string,"SBM_string": SBM_string}
        regions.append((df.Filter("{r} && DiLepMass>54 && {RelMET_string}<{RelMET} && {SBM_string}>{SBM}".format(**format_dict), region_string), format_dict))

def process_sample(sp,sample,era,verbose=1):
    #get metadata
    name,xsec,nevts = sample['name'],sample['xsec'],sample['nevts']
    ismc,fileglob,bTagEff = int(sample['ismc']),sample['fileglob'],sample['bTagEff']
    if not nevts:
        nevts = get_nEvents(fileglob)
    sample_weight = float(xsec)*sp.lumi/float(nevts)
    if verbose: print("name: {} , xsec: {}, nevents: {} ismc: {}".format(name,xsec,nevts,ismc))
    #make file glob
    files = toVector('string', glob(fileglob))
    #set up btagging and puid sf files
    bTagFile, PUIDSFfile = setup_btag_puid(ismc, era, bTagEff)
    #make rdf
    df = RDFrame('Events', files)
    df = df.Filter("DiLepMass>54", "mass_cut")
    df = df.Filter("SR1_nominal or CR10_nominal or CR11_nominal or CR12_nominal or CR13_nominal or CR14_nominal or SR2_nominal or CR20_nominal or CR21_nominal or CR22_nominal or CR23_nominal or CR24_nominal", "in_region")
    df = def_good_jet(df,ismc, bDiscValue)
    df = def_good_leptons(df, ismc)
    df = def_HLT(df, ismc, era)
    df = def_sf_and_weight(df,ismc, sp.is_inclusive, name, sample_weight)
    df = def_lep_selections(df)
    columns = ['Weight','sample_weight','HTLT_nominal','RelMET_nominal','SBM_nominal','SR2_nominal','DiLepMass',
        'SR1_nominal','CR10_nominal','CR11_nominal','CR12_nominal','CR13_nominal','CR14_nominal','SR2_nominal','CR20_nominal','CR21_nominal','CR22_nominal','CR23_nominal','CR24_nominal', 
               'TriggerWeight', "Flag_goodVertices","Flag_globalSuperTightHalo2016Filter","Flag_HBHENoiseFilter","Flag_HBHENoiseIsoFilter","Flag_EcalDeadCellTriggerPrimitiveFilter","Flag_BadPFMuonFilter","Flag_eeBadScFilter","Flag_METFilters"]
    df.Snapshot("Events", "data/tw_{}_{}.root".format(era,name), columns)
    df_np = df.AsNumpy(columns)
    df_df = pd.DataFrame(df_np)
    df_df.to_csv('data/tw_{}_{}.csv'.format(era,name))
    return name,df, fileglob

In [17]:
#####
##### BTag SF not included
#####
for sample in sp.samples():
    #if 'data' not in sample['name']: continue 
    start_time = perf_counter()
    name,df,fileglob = process_sample(sp,sample,era)
    #count = df.Count()
    end_time = perf_counter()
    try:
        print(name,end_time-start_time)
    except:
        print(name,end_time-start_time)
sp.close()

name: ZToEE_M_120_200 , xsec: 19320.0, nevents: 100000.0 ismc: 1
ZToEE_M_120_200 7.259641842916608
name: ZToEE_M_200_400 , xsec: 2731.0, nevents: 99200.0 ismc: 1
ZToEE_M_200_400 6.037593415006995
name: ZToEE_M_400_800 , xsec: 241.0, nevents: 100000.0 ismc: 1
ZToEE_M_400_800 11.593968365341425
name: ZToEE_M_50_120 , xsec: 1975000.0, nevents: 2998400.0 ismc: 1
ZToEE_M_50_120 7.591890912503004
name: ZToEE_M_800_1400 , xsec: 16.78, nevents: 100000.0 ismc: 1
ZToEE_M_800_1400 8.981179296970367
name: ZToMuMu_M_120_200 , xsec: 19320.0, nevents: 100000.0 ismc: 1
ZToMuMu_M_120_200 5.2276714984327555
name: ZToMuMu_M_200_400 , xsec: 2731.0, nevents: 100000.0 ismc: 1
ZToMuMu_M_200_400 6.985632576048374
name: ZToMuMu_M_400_800 , xsec: 241.0, nevents: 98400.0 ismc: 1
ZToMuMu_M_400_800 9.064283084124327
name: ZToMuMu_M_50_120 , xsec: 1975000.0, nevents: 100000.0 ismc: 1
ZToMuMu_M_50_120 5.825346248224378
name: ZToMuMu_M_800_1400 , xsec: 16.78, nevents: 100000.0 ismc: 1
ZToMuMu_M_800_1400 11.5400103162

In [9]:
['sample_weight', 'HTLT', 'RelMET', 'SBM', 'SR2', 'DiLepMass', 'SR1', ]

['sample_weight', 'HTLT', 'RelMET', 'SBM', 'SR2', 'DiLepMass', 'SR1']

In [10]:
for x in df.GetColumnNames():
    print(x)

GoodJet
GoodJetPt
GoodJetEta
nJets
BJet
GoodBJet
leading_b_jet_pt
leading_jet_pt
NoPUID_GoodJet
NoPUID_GoodJetPt
NoPUID_GoodJetEta
NoPUID_PUID
NoPUID_GoodJetGenJetIdx
GoodMuon
GoodMuonPt
GoodMuonEta
GoodMuonPhi
GoodMuonCharge
nMuons
GoodMuonLowPt
GoodMuonPtLow
GoodMuonEtaLow
nMuonsLowPt
GoodElectron
GoodElePt
GoodEleEta
GoodElePhi
GoodEleCharge
nEle
GoodElectronLowPt
GoodElePtLow
GoodEleEtaLow
nEleLowPt
GoodMuon_effSF_trigger
GoodMuon_effSF_stat_trigger
GoodMuon_effSF_ID
GoodMuon_effSF_sys_ID
GoodMuon_effSF_stat_ID
GoodMuon_effSF_ISO
GoodMuon_effSF_sys_ISO
GoodMuon_effSF_stat_ISO
GoodElectron_effSF
GoodElectron_effSF_stat
GoodElectron_effSF_sys
TriggerRegion1
TriggerRegion2
TriggerRegion3
TriggerWeight
PDF_ISRFSR_uncertainty
PDF_ISRFSR_uncertaintyUp
PDF_ISRFSR_uncertaintyDown
MuonSFweights
MuonSFweight
MuonSFweightUp
MuonSFweightDown
ElectronSFweights
ElectronSFweight
ElectronSFweightUp
ElectronSFweightDown
k_factor
PUIDWeights
PUIDWeight
PUIDWeightUp
PUIDWeightDown
GoodJetBTagSF
GoodJ